In [69]:
aoi2 = {
            "type": "Feature",
            "properties": {
                "name": "포항 호미곶"
            },
            "geometry": {
                "type": "Polygon",
                "coordinates": [
                    [
                        [
                            129.56524548895032,
                            36.07938410532303
                        ],
                        [
                            129.56655182188928,
                            36.07905999622248
                        ],
                        [
                            129.56894545770393,
                            36.07834416441952
                        ],
                        [
                            129.5700954118841,
                            36.077591279361485
                        ],
                        [
                            129.56983938956853,
                            36.07690633718157
                        ],
                        [
                            129.56985996844412,
                            36.07651088998368
                        ],
                        [
                            129.57148025132796,
                            36.07560318895601
                        ],
                        [
                            129.57242351742929,
                            36.076299330209935
                        ],
                        [
                            129.57336311237833,
                            36.08009412639197
                        ],
                        [
                            129.57095418243125,
                            36.08296641858419
                        ],
                        [
                            129.5665347381402,
                            36.082332370616356
                        ],
                        [
                            129.56509566557435,
                            36.08086763810617
                        ],
                        [
                            129.56524548895032,
                            36.07938410532303
                        ]
                    ]
                ]
            }
        }

In [70]:
import ee
import pandas as pd
from prophet import Prophet
import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots

In [72]:
ee.Authenticate()


Successfully saved authorization token.


In [79]:
ee.Initialize()

In [80]:
def create_ee_polygon_from_geojson(gjson):
    coordinates = gjson['geometry']['coordinates']
    aoi = ee.Geometry.Polygon(coordinates)
    return aoi

In [89]:
def calculateNDVI(aoi, start_date, end_date):
    # Sentinel-2 ImageCollection 필터링 및 구름 마스킹 적용
    sentinel2 = ee.ImageCollection('COPERNICUS/S2') \
            .filterBounds(aoi) \
            .filterDate(start_date, end_date) \
            .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10)) 

    # NDVI 계산 및 시계열 데이터 생성 함수
    def calculate_ndvi(image):
        date = ee.Date(image.get('system:time_start')).format('YYYY-MM-dd')
        nir = image.select('B8')  # NIR 밴드
        red = image.select('B4')  # Red 밴드
        ndvi = nir.subtract(red).divide(nir.add(red)).rename('ndvi')
        mean_ndvi = ndvi.reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=aoi,
            scale=10  # 이 값은 필요에 따라 조정할 수 있습니다.
        ).get('ndvi')
        return ee.Feature(None, {'ds': date, 'y': mean_ndvi})
    
    
    time_series_ndvi = sentinel2.map(calculate_ndvi)
    
    # 결과를 서버측 객체로 변환 (Python 클라이언트로 가져오기 위함)
    rvi_features = time_series_ndvi.getInfo()['features']

    # 결과를 pandas DataFrame으로 변환
    df = pd.DataFrame([feat['properties'] for feat in rvi_features])

    # DataFrame을 'Date' 컬럼에 따라 오름차순으로 정렬
    df = df.sort_values(by='ds')
    
    return df

In [90]:
aoi = create_ee_polygon_from_geojson(aoi2)

In [91]:
start_date = '2013-01-01'
end_date = '2023-10-31'

In [92]:
df = calculateNDVI(aoi,start_date,end_date)

EEException: Error in map(ID=20170430T020701_20170430T021509_T52SEE):
Image.select: Parameter 'input' is required.

In [64]:
df.describe()

,y
count,972.000000
mean,-0.064836
std,0.083554
min,-0.336028
25%,-0.121905
50%,-0.033221
75%,-0.003711
max,0.457788


In [65]:
def prophet_process(df):
    # Prophet 모델을 초기화하고 학습시킵니다.
    m = Prophet(yearly_seasonality=True)
    m.fit(df)

    # 미래 날짜 프레임을 만들고 예측을 진행합니다.
    future = m.make_future_dataframe(periods=12,freq='M')
    forecast = m.predict(future) 

    # 예측 결과를 가져옵니다.
    forecasted_value = forecast.iloc[-1]['yhat']  # 예측된 값을 가져옴
    # 예측 결과를 데이터프레임에 추가합니다.
    forecast_df = df.append({'ds': future.iloc[-1]['ds'], 'y': forecasted_value}, ignore_index=True)
    return forecast,forecast_df,df,m

def plotly(df, forecast):
    # Create a Plotly Express figure for both forecast and observed data
    combined_fig = px.line(forecast, x='ds', y='yhat', title='예측')
    
    # Add observed data to the same figure
    combined_fig.add_trace(px.scatter(df, x='ds', y='y', title='관측치', color_discrete_sequence=['red']).data[0])

    # Display the combined figure using st.plotly_chart()
    st.plotly_chart(combined_fig, use_container_width = True)